# Analyze NYC Taxi Data to Improve Transportation Policy
🎯 Goal: Help city planners understand how trip distances, durations, and demand vary across time and location to improve infrastructure or pricing policy.

### 📦 Dataset: NYC Yellow Taxi Trips

Link: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

Format: Parquet

Also download the Zone Lookup Table to map *locations*

In [2]:
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-08.parquet --output ./yellow_tripdata_2025-08.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 59.4M  100 59.4M    0     0   117M      0 --:--:-- --:--:-- --:--:--  117M


In [3]:
!curl https://d37ci6vzurychx.cloudfront.net/misc/taxi+_zone_lookup.csv --output ./taxi_zone_lookup.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12331  100 12331    0     0   116k      0 --:--:-- --:--:-- --:--:--  116k


### STEP 1: Load Taxi Data and Zone Info

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("NYCTaxiAnalysis").getOrCreate()

# Load the data
trips = spark.read.parquet("./yellow_tripdata_2025-08.parquet")

# Load Lookup table
zones = spark.read.csv("./taxi_zone_lookup.csv", header=True, inferSchema=True)

### STEP 2: Clean and Prepare the Data

In [ ]:
from pyspark.sql.functions import col, unix_timestamp

# Remove invalid records
trips_clean = trips.filter(
    (col('passenger_count') > 0) &
    (col('trip_distance') > 0) &
    (col('tpep_dropoff_datetime') > col('tpep_pickup_datetime'))
)

# Calculate the trip duration in minutes
trips_clean = trips_clean.withColumn(
    "trip_duration_min",
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60
)

### STEP 3: Join with Zone Data (to get readable zone names)

In [ ]:
# Rename for clarity
zones = zones.withColumnRenamed("LocationID", "PULocationID").withColumnRenamed("Zone", "PU_Zone")

# Join to get pickup zone names
trips_with_zones = trips_clean.join(zones, on="PULocationID", how='left')

### Core Analysis

In [ ]:
# Top 10 Pickup Zones by Trip Count
trips_with_zones.groupBy("PU_Zone")\
  .count()\
  .orderBy("count", ascending=False)\
  .show(10)

+--------------------+------+
|             PU_Zone| count|
+--------------------+------+
|         JFK Airport|183342|
|      Midtown Center|123687|
|Upper East Side S...|111222|
|Penn Station/Madi...|106469|
|   LaGuardia Airport| 97109|
|        Midtown East| 94813|
|Times Sq/Theatre ...| 89936|
|Upper East Side N...| 85777|
|         Murray Hill| 79674|
|       Midtown North| 71146|
+--------------------+------+
only showing top 10 rows



In [ ]:
from pyspark.sql.functions import hour

trips_with_hours = trips_with_zones.withColumn("pickup_hour", hour("tpep_pickup_datetime"))

# Avg Trip Duration by Hour of Day
trips_with_hours.groupBy("pickup_hour")\
  .avg("trip_duration_min")\
  .orderBy('pickup_hour')\
  .show()

+-----------+----------------------+
|pickup_hour|avg(trip_duration_min)|
+-----------+----------------------+
|          0|    15.200350119364197|
|          1|    13.972758818285994|
|          2|    12.714887250363725|
|          3|    13.009176058874996|
|          4|     15.63602369255961|
|          5|     18.89007348400631|
|          6|    17.573079565951776|
|          7|    17.254840255331587|
|          8|     16.54589741268411|
|          9|     16.88826175396414|
|         10|      17.2379948556447|
|         11|    17.800190142042112|
|         12|    18.373015284029513|
|         13|    18.489530655981863|
|         14|    19.889117733881196|
|         15|    19.860048822492594|
|         16|    19.924008877162485|
|         17|    18.844386985384023|
|         18|     16.86838233447732|
|         19|     16.01068547549598|
+-----------+----------------------+
only showing top 20 rows



In [5]:
# Short Distance, Long Duration Trips
trips_with_zones.filter(
  (col("trip_distance") < 2) & (col("trip_duration_min") > 20))\
  .select("PU_Zone", "trip_distance", "trip_duration_min")\
  .orderBy("trip_duration_min", ascending=False)\
  .show(10)

NameError: name 'trips_with_zones' is not defined

These could be traffic-congested areas — worth investigating for infrastructure or traffic light optimization.

In [ ]:
trips_with_zones.write.parquet("cleaned_nyc_taxi.parquet")

#save analysis summary
top_zones = trips_with_zones.groupBy("PU_Zone")\
  .count()\
  .orderBy("count", ascending=False)

In [ ]:
top_zones.write.csv("top_pickup_zones.csv", header=True)